## Loading required libraries

In [1]:
import cv2
import numpy as np
from os import path

# %matplotlib notebook
# %matplotlib inline
%matplotlib qt
# bugged in win
# %matplotlib tk 
import matplotlib.pyplot as plt

## Resizing and Cropping

### Resizing

In [7]:
img = cv2.imread(r"./data/lena.jpg")
print(f"Image Shape: {img.shape}")
cv2.imshow("Image", img)

resized_img = cv2.resize(img, (256, 256))
print(f"Image Shape: {resized_img.shape}")
cv2.imshow("Resized Image", resized_img)

cv2.waitKey(0)
cv2.destroyAllWindows()

Image Shape: (512, 512, 3)
Image Shape: (256, 256, 3)


### Cropping

In [8]:
img = cv2.imread(r"./data/lena.jpg")
cv2.imshow("Image", img)

cropped_img = img[0:256, :256]
cv2.imshow("Cropped Image", cropped_img)

cv2.waitKey(0)
cv2.destroyAllWindows()

Image Shape: (512, 512, 3)


## Warp Perspective

In [10]:
img = cv2.imread(r"./data/cards.jpg")
cv2.imshow("Image", img)

# extracting king of spades
width, height = 250, 350
pts1 = np.float32([[111,219],[287,188],[154,482],[352,440]])  # points on original img
pts2 = np.float32([[0,0],[width,0],[0,height],[width,height]])  # points reference for transformation
matrix = cv2.getPerspectiveTransform(pts1, pts2)
img_out = cv2.warpPerspective(img, matrix, (width, height))
cv2.imshow("Card", img_out)

cv2.waitKey(0)
cv2.destroyAllWindows()

## Merging Images

In [2]:
img = cv2.imread(r"./data/lena.jpg")
cv2.imshow("Image", img)

stackedH = np.hstack((img, img))
cv2.imshow("stackedH", stackedH)

stackedV = np.vstack((img, img))
cv2.imshow("stackedV", stackedV)

cv2.waitKey(0)
cv2.destroyAllWindows()

### Stack Images

In [4]:
# Helper function to scale and stack images
def stackImages(scale, imgArray):
    rows = len(imgArray)
    cols = len(imgArray[0])
    rowsAvailable = isinstance(imgArray[0], list)
    width = imgArray[0][0].shape[1]
    height = imgArray[0][0].shape[0]
    if rowsAvailable:
        for x in range(0, rows):
            for y in range(0, cols):
                if imgArray[x][y].shape[:2] == imgArray[0][0].shape[:2]:
                    imgArray[x][y] = cv2.resize(
                        imgArray[x][y], (0, 0), None, scale, scale
                    )
                else:
                    imgArray[x][y] = cv2.resize(
                        imgArray[x][y],
                        (imgArray[0][0].shape[1], imgArray[0][0].shape[0]),
                        None,
                        scale,
                        scale,
                    )
                if len(imgArray[x][y].shape) == 2:
                    imgArray[x][y] = cv2.cvtColor(imgArray[x][y], cv2.COLOR_GRAY2BGR)
        imageBlank = np.zeros((height, width, 3), np.uint8)
        hor = [imageBlank] * rows
        hor_con = [imageBlank] * rows
        for x in range(0, rows):
            hor[x] = np.hstack(imgArray[x])
        ver = np.vstack(hor)
    else:
        for x in range(0, rows):
            if imgArray[x].shape[:2] == imgArray[0].shape[:2]:
                imgArray[x] = cv2.resize(imgArray[x], (0, 0), None, scale, scale)
            else:
                imgArray[x] = cv2.resize(
                    imgArray[x],
                    (imgArray[0].shape[1], imgArray[0].shape[0]),
                    None,
                    scale,
                    scale,
                )
            if len(imgArray[x].shape) == 2:
                imgArray[x] = cv2.cvtColor(imgArray[x], cv2.COLOR_GRAY2BGR)
        hor = np.hstack(imgArray)
        ver = hor
    return ver

In [6]:
img1 = cv2.imread(r"./data/cards.jpg")
print(f"Img1 shape: {img1.shape}")
cv2.imshow("img1", img1)

img2 = cv2.imread(r"./data/lena.jpg")
print(f"Img2 shape: {img2.shape}")
cv2.imshow("img2", img2)

cv2.waitKey(0)
cv2.destroyAllWindows()

Img1 shape: (500, 477, 3)
Img2 shape: (512, 512, 3)


In [9]:
satckedImages = stackImages(scale=0.5, imgArray=([img1, img2, img1], [img2, img1, img2]))
cv2.imshow("Stacked Images", satckedImages)

cv2.waitKey(0)
cv2.destroyAllWindows()

## Color Detection

In [12]:
img = cv2.imread(r"./data/lambo.png")
imgHSV = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

def empty(X):
    pass

cv2.namedWindow("TrackBars")
cv2.resizeWindow("TrackBars", 640, 240)
cv2.createTrackbar("Hue Min", "TrackBars", 0, 179, empty) # hue max is 179
cv2.createTrackbar("Hue Max", "TrackBars", 19, 179, empty)

cv2.createTrackbar("Sat Min", "TrackBars", 110, 255, empty)
cv2.createTrackbar("Sat Max", "TrackBars", 240, 255, empty)

cv2.createTrackbar("Val Min", "TrackBars", 153, 255, empty)
cv2.createTrackbar("Val Max", "TrackBars", 255, 255, empty)

while True:
    h_min = cv2.getTrackbarPos("Hue Min", "TrackBars")
    h_max = cv2.getTrackbarPos("Hue Max", "TrackBars")
    s_min = cv2.getTrackbarPos("Sat Min", "TrackBars")
    s_max = cv2.getTrackbarPos("Sat Max", "TrackBars")
    v_min = cv2.getTrackbarPos("Val Min", "TrackBars")
    v_max = cv2.getTrackbarPos("Val Max", "TrackBars")
    
    lower = np.array([h_min, s_min, v_min])
    upper = np.array([h_max, s_max, v_max])
    mask = cv2.inRange(imgHSV, lower, upper)
    
    imgResult = cv2.bitwise_and(img, img, mask=mask)
    
    satckedImages = stackImages(scale=0.6, imgArray=([img, imgHSV], [mask, imgResult]))
    cv2.imshow("Mask", satckedImages)
    #cv2.imshow("HSV", imgHSV)
    #cv2.imshow("img1", img)
    #cv2.imshow("Mask", mask)
    #cv2.imshow("Result", imgResult)

    if cv2.waitKey(1) & 0xFF == 27:
        break
        
cv2.destroyAllWindows()

## Contour/ Shape Detection

### get Contours

In [2]:
def getContours(img, orig_img):
    imgContour = orig_img.copy()
    contours, hierarchy = cv2.findContours(
        img,
        mode=cv2.RETR_EXTERNAL,  # contour retrieval mode: get extreme outer contour
        method=cv2.CHAIN_APPROX_NONE,  # Whether compressed values or all points: all
    )
    for contour in contours:
        area = cv2.contourArea(contour)
        # print(area)

        if area > 500:  # only draw contours having area > 500 pixels
            cv2.drawContours(imgContour, contour, -1, (255, 0, 0), 3)

            # calxulate curve length, used to approximate couners of image
            peri = cv2.arcLength(contour, closed=True)
            # print(peri)  # perimeter of contour

            # approximate corner points, gives corner points
            approx = cv2.approxPolyDP(curve=contour, epsilon=0.02 * peri, closed=True)
            # print(len(approx))

            objCor = len(approx)
            x, y, w, h = cv2.boundingRect(approx)

            if objCor == 3:
                objectType = "Tri"
            elif objCor == 4:
                aspRatio = w / float(h)
                if aspRatio > 0.98 and aspRatio < 1.03:
                    objectType = "Square"
                else:
                    objectType = "Rectangle"
            elif objCor > 4:
                objectType = "Circles"
            else:
                objectType = "None"

            cv2.rectangle(imgContour, (x, y), (x + w, y + h), (0, 255, 0), 2)
            cv2.putText(
                imgContour,
                objectType,
                (x + (w // 2) - 10, y + (h // 2) - 10),
                cv2.FONT_HERSHEY_COMPLEX,
                0.7,
                (0, 0, 0),
                2,
            )
    return imgContour

In [5]:
img = cv2.imread(r"./data/shapes2.png")

imgBlank = np.zeros_like(img)
imgray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
imgBlur = cv2.GaussianBlur(imgray, (7, 7), 1)
imgCanny = cv2.Canny(imgBlur, 50, 50)
drawnImage = getContours(imgCanny, img)

stackedImages = stackImages(0.6, imgArray=([img, imgray, imgBlur], [imgCanny, drawnImage, imgBlank]))
cv2.imshow("Stacks", stackedImages)

cv2.waitKey(0)
cv2.destroyAllWindows()

## Face Detection

In [5]:
img = cv2.imread(r"./data/lena.jpg")

faceCascade = cv2.CascadeClassifier(r"./data/haarcascade_frontalface_default.xml")
imgray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

faces = faceCascade.detectMultiScale(imgray, scaleFactor=1.1, minNeighbors=4)

for (x, y, w, h) in faces:
    cv2.rectangle(img, (x, y), (x + w, y + h), (255, 0, 0), 2)

cv2.imshow("Lena", img)
cv2.waitKey(0)
cv2.destroyAllWindows()